# **Imputar preprocesado de forma aleatoria y Random Forest**

Se reemplazar los datos de forma aleatoria por un dato existente en la columna y se utiliza Random Forest para predecir

# **IMPORTACIÓN**

Se importan y descargaran cada una de las librerias que se utilizaran

In [2]:
!pip install rlxutils

  Preparing metadata (setup.py) ... done
  Created wheel for rlxutils: filename=rlxutils-0.1.10-py3-none-any.whl size=11100 sha256=07cf8fca490921be89ab69e8aa587f2ba3d876e2bbb096d0ee9f04bcee209c60
  Stored in directory: /root/.cache/pip/wheels/9a/45/da/49bdb0e82cc7a605e9c05dc24265687c5f349b53dcb74728e2
Successfully built rlxutils


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import gdown
from rlxutils import subplots

# **Drive**

Se conecta con el drive para descargar los archivos del mismo y cargarlos al coolab para manejarlos

In [4]:
url1 = f'https://drive.google.com/uc?id=1EFM-jnbWsT2buCk6NjT_XAffz0oeVyLE'
url2 = f'https://drive.google.com/uc?id=1bUsh5xZo8X6Gu9vnqHsrZIyd54XkcZSv'
gdown.download(url1, 'train.csv', quiet=False)
gdown.download(url2, 'test.csv', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1EFM-jnbWsT2buCk6NjT_XAffz0oeVyLE
From (redirected): https://drive.google.com/uc?id=1EFM-jnbWsT2buCk6NjT_XAffz0oeVyLE&confirm=t&uuid=dd8c629f-c043-4308-bd68-f0d53d8e6eb8
To: /content/train.csv
100%|██████████| 118M/118M [00:01<00:00, 72.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bUsh5xZo8X6Gu9vnqHsrZIyd54XkcZSv
To: /content/test.csv
100%|██████████| 50.1M/50.1M [00:00<00:00, 167MB/s]


'test.csv'

In [5]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# **Función para reemplazar los datos**

Recibe el dataframe y en nombre de la columna a la cual se va a reemplazar y para todos los valores nulos los reemplaza por un valor aleatoria de la misma columna

In [6]:
# Función para reemplazar valores nulos con valores aleatorios de la misma columna
def fill_na_with_random_value(df, column):
    non_null_values = df[column].dropna().values
    df[column] = df[column].apply(lambda x: np.random.choice(non_null_values) if pd.isna(x) else x)

# **Funcion clean_data**



1.   Se eliminan las columnas no necesarias o repetidas
2.   Se reemplazan los datos faltantes de forma aleatoria
3.   Se normalizan los valores en varias columnas
4.   Se hace una normalización especifica pala la columna ESTU_PRGM_ACADEMICO












In [7]:
def clean_data(df):

  columnas_con_nulos = df.columns[df.isnull().any()].tolist()

  for col in columnas_con_nulos:
    fill_na_with_random_value(df, col)

  #normalización de valores
  df['FAMI_ESTRATOVIVIENDA'].replace(['Estrato 1','Estrato 2','Estrato 3','Estrato 4','Estrato 5','Estrato 6','Sin Estrato'],[1,2,3,4,5,6,0],inplace=True)
  #df['RENDIMIENTO_GLOBAL'].replace(['bajo','medio-bajo','medio-alto','alto'],[1,2,3,4],inplace=True)

  df["FAMI_TIENEINTERNET"] = df["FAMI_TIENEINTERNET"].replace({'Si': 1, 'No': 0})
  df["ESTU_PAGOMATRICULAPROPIO"] = df["ESTU_PAGOMATRICULAPROPIO"].replace({'Si': 1, 'No': 0})

  df['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['No pagó matrícula'],[0],inplace=True)
  df['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['Menos de 500 mil'],[1],inplace=True)
  df['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['Entre 500 mil y menos de 1 millón'],[2],inplace=True)
  df['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['Entre 1 millón y menos de 2.5 millones'],[3],inplace=True)
  df['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['Entre 2.5 millones y menos de 4 millones'],[4],inplace=True)
  df['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['Entre 4 millones y menos de 5.5 millones'],[5],inplace=True)
  df['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['Entre 5.5 millones y menos de 7 millones'],[6],inplace=True)
  df['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['Más de 7 millones'],[7],inplace=True)

  df['ESTU_HORASSEMANATRABAJA'].replace(['0','Menos de 10 horas','Entre 11 y 20 horas','Entre 21 y 30 horas','Más de 30 horas'],[0,1,2,3,4],inplace=True)


  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL EN ', '')

  #df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'INGENIERIA' if x.startswith('INGE') else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'LICENCIATURA' if x.startswith('LIC') or x.startswith('Lic') else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ADMINISTRACION' if x.startswith('ADM')   else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CONTADURIA' if x.startswith('CONT')   else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'COMUNICACION' if x.startswith('COMU')   else x)

  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'DEPORTE' if 'DEPORT'  in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'PSICOLOGIA' if 'PSIC' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'TRABAJO SOCIAL' if 'TRABAJO SOCIAL' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'DISEÑO' if 'DISE' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'TERAPIA' if 'TERAP' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'GASTRONOMIA' if 'GASTRON' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'VETERINARIA Y ZOOTECNIA' if 'ZOOTECNIA' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'NUTRICION' if 'NUTRIC' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'MARKETING' if ( 'MARKETING' in x or 'PUBLICIDAD' in x or 'MERCAD' in x) else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'MATEMATICAS Y ESTADISTICA' if ('MATE' in x or 'ESTA' in x) else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ARTES' if 'ARTE' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'LITERATURA' if 'LITER' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'MUSICA' if ('MUS' in x or 'MÚS' in x) else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ECONOMIA Y NEGOCIOS' if ('ECONO' in x or 'NEG' in x or 'COME' in x or 'FIN' in x) else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CIENCIAS DE LA TIERRA' if 'GEO' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CIENCIAS AUDIOVISUALES' if ('CINE' in x or 'RADIO' in x or 'TELEV' in x or 'AUDIOV' in x or 'FOTOG' in x) else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'DERECHO' if 'DERECHO' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CIENCIAS POLITICAS' if ( 'POLIT' in x or 'POLÍT' in x) else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'INGENIERIA' if 'INGE' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'QUIMICA' if ('QUIM' in x or 'QUÍM' in x or 'QU¿M' in x) else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CONSTRUCCION' if 'CONST' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'TURISMO' if 'TURIS' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'BIBLIOTECOLOGIA' if 'BIBLIO' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'FILOSOFIA' if 'FILOS' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'FILOLOGIA' if 'FILOL' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ODONTOLOGIA' if 'ODONTOLOG' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'BIOLOGIA Y MICROBIOLOGIA' if 'BIOL' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ECOLOGIA' if x.startswith('ECOL')   else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'BACTERIOLOGIA' if 'BACT' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].replace('ACUICULTURA', 'ACUICUlTURA')
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CIENCIAS CULTURALES' if 'CULT' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'INSTRUMENTACION QUIRURGICA' if 'INSTRUMENTACION' in x else x)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CIENCIAS MARITIMAS Y AERONAUTICAS' if ('MILIT' in x or 'NAVALES' in x or 'NAU' in x) else x)


  # Calcula el conteo de cada valor en la columna 'ESTU_PRGM_ACADEMICO'
  conteo_valores = df['ESTU_PRGM_ACADEMICO'].value_counts()

  # Obtén los valores que tienen menos de N registros
  valores_menos_de_20 = conteo_valores[conteo_valores <= 20]
  valores_menos_de_20

  # Reemplaza los valores que tienen menos de N registros con 'OTRO PROGRAMA'
  for valor in valores_menos_de_20.index:
      df.loc[df['ESTU_PRGM_ACADEMICO'] == valor, 'ESTU_PRGM_ACADEMICO'] = 'OTRO PROGRAMA'

  df = pd.get_dummies(df, columns= ["ESTU_PRGM_ACADEMICO","ESTU_PRGM_DEPARTAMENTO","FAMI_EDUCACIONPADRE","FAMI_EDUCACIONMADRE"])
  df = df.fillna(-1).astype(int)



  return df

In [8]:
lentr = len(df_train)

# Definir las columnas de origen, excluyendo la variable objetivo
source_cols = [i for i in df_train.columns if i != "RENDIMIENTO_GLOBAL"]

# Combinar los datos de entrenamiento y prueba para preprocesamiento
all_data = pd.concat((df_train[source_cols], df_test[source_cols]))
all_data.index = range(len(all_data))

# Limpiar los datos usando la función clean_data
all_data = clean_data(all_data)

# Dividir los datos combinados nuevamente en conjuntos de entrenamiento y prueba
Xtrain = all_data.iloc[:lentr].values
ytrain = df_train["RENDIMIENTO_GLOBAL"].values
Xtest = all_data.iloc[lentr:].values

# Imprimir las formas de los conjuntos de entrenamiento y prueba
print("Forma de Xtrain:", Xtrain.shape)
print("Forma de ytrain:", ytrain.shape)
print("Forma de Xtest:", Xtest.shape)

<ipython-input-7-b3c18f7febc4>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['FAMI_ESTRATOVIVIENDA'].replace(['Estrato 1','Estrato 2','Estrato 3','Estrato 4','Estrato 5','Estrato 6','Sin Estrato'],[1,2,3,4,5,6,0],inplace=True)
<ipython-input-7-b3c18f7febc4>:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcastin

Forma de Xtrain: (692500, 162)
Forma de ytrain: (692500,)
Forma de Xtest: (296786, 162)


# **Entrenar el modelo**

In [9]:
# Importar las bibliotecas necesarias
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

# Definir el clasificador RandomForest
#rf = RandomForestClassifier()

# Realizar validación cruzada en el conjunto de entrenamiento
#cv_scores = cross_val_score(rf, Xtrain, ytrain)
#print("Puntuaciones de validación cruzada:", cv_scores)

rf = RandomForestClassifier(n_estimators=75, max_depth=8, min_samples_split=3, max_features='sqrt')
print (cross_val_score(rf, Xtrain, ytrain, cv=5)) # cv=5 para validación cruzada con 5 pliegues


# Entrenar el clasificador en los datos de entrenamiento
rf.fit(Xtrain, ytrain)

# Predecir en el conjunto de prueba
preds_ts = rf.predict(Xtest)
print("Predicciones en el conjunto de prueba:", preds_ts)

[0.38755235 0.38766065 0.38987004 0.38765343 0.38764621]
Predicciones en el conjunto de prueba: ['bajo' 'bajo' 'alto' ... 'alto' 'alto' 'alto']


# **Creación de archivo con las variables necesarias**

In [10]:
# Crear un DataFrame de envío con el ID y los valores predichos
submission = pd.DataFrame({
    "ID": df_test.ID,
    "RENDIMIENTO_GLOBAL": preds_ts
})

# Mostrar las primeras filas del DataFrame de envío
print(submission.head())

# Guardar el DataFrame de envío en un archivo CSV
print("*************** Crear archivo ************")
submission.to_csv("archivo_final.csv", index=False)

       ID RENDIMIENTO_GLOBAL
0  550236               bajo
1   98545               bajo
2  499179               alto
3  782980               bajo
4  785185               bajo
*************** Crear archivo ************
